# TO ADD

- https://kubernetes.io/docs/concepts/services-networking/connect-applications-service/ - example using command line and wondering around (mixed service and application)... Maybe should be done after `kubectl`?
- Describe `Ingress` (access to Kubernetes cluster outside the cluster)
- Ingress vs LoadBalancers and NodePorts: https://platform9.com/blog/understanding-kubernetes-loadbalancer-vs-nodeport-vs-ingress/


> Network policies: https://kubernetes.io/docs/concepts/services-networking/network-policies/

# Networking & Services

Networking capabilities addresses:
- __Containers networking within `POD`__ use `loopback` for communication
- __Cluster networking between different `POD`s__
- __`Service` resource__ - exposes application running in `POD`s __to the outside world__ (or for within cluster consumption)

> `loopback` routes signal back to the source (in our case from the same `IP` to the same `IP` address)

# Service

> __An abstract way to expose an application running on a set of Pods as a network service__

Below is an example motivation for `Service`'s existence:

- Each `POD` has it's own IP address
- `POD`s are dynamically created and deleted (say, via `Deployment` resource workload)
- How to communicate with this dynamically change'able `set`?

> __Service defines a logical set of `POD`s and a `policy` to access them__

## Defining

As per usual, `Service` is defined via posting REST request to `kube-apiserver` (yeah, using `.yaml` config all in all and `kubelet`):

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app: MyApp
  ports:
    - protocol: TCP
      targetPort: 9376
      port: 80

There are a few distinctions between `Service` and `Workload resource`s we saw last time:
- __`Service` does not create `POD`s__
- __`Service` manages `POD`s__ which are matched based on selector
- new `ports` field

### `ports`

> `ports` specify how to forward requests to `POD`s matching `selector`

In this case:
- Use `tcp` protocol
- Forward requests to `POD`s port `9376`
- Make `Service` itself available under port `80`

Kubernetes will assign `IP` address to this service using which we can communicate with all of the matching `POD`s (these are continuously scanned)

> One can specify multiple `ports` for other network components to query

Example of multiple `ports` (`name` is used to disambiguate between `port`s):

In [ ]:
---
kind: Service
apiVersion: v1
metadata:
  name: my-service
spec:
  selector:
    app: MyApp
  ports:
  - name: management 
    # TCP is default protocol
    port: 80
    targetPort: 8000
  - name: query 
    protocol: TCP
    port: 81
    targetPort: 8001

For each `Service`'s defined `port` new `Endpoint` k8s object is defined implicitly.

> Maximum number of `endpoints` (ports in this case) for `service` is `1000`, otherwise __these are truncated__

Example `Endpoint` configuration can be seen below:

In [ ]:
apiVersion: v1
kind: Endpoints
metadata:
  name: my-service
subsets:
  - addresses:
      - ip: 192.0.2.42
    ports:
      - port: 9376

## Endpoint Slices

> Solution to scaling number of `Endpoint`s for ever growing services

As `Endpoints` were getting larger each change would inflict a lot of work on `control plane` of Kubernetes.

`EndpointSlice`s also allow us to define topological routing (described later) and __are automatically created for `k8s` `Service` with `.spec.selector`__.

> __`Endpoint Slices` group endpoints together based on unique combination of `protocol`, `port` and `Service name`!__

Defining `EndpointSlice` can be done as described below:

In [ ]:
apiVersion: discovery.k8s.io/v1
kind: EndpointSlice
metadata:
  name: example-abc
  labels:
    kubernetes.io/service-name: example
addressType: IPv4
ports:
  - name: http
    protocol: TCP
    port: 80
endpoints:
  - addresses:
      - "10.1.2.3"
    conditions:
      ready: true
    hostname: pod-1
    nodeName: node-1
    zone: us-west2-a

- address types: `IPv4`, `IPv6` or `Fully Qualified Domain Name`
- conditions: info about grouped `endpoints`, one of:
    - `ready` - meaning `Endpoint` is ready to serve request
    - `serving`  - the same, __but also fine when `Endpoint` is terminating but still can take `request`__
    - `terminating`
- __topology information__:
    - `nodeName`: name of node the endpoint is on
    - `zone`: zone `Node` is in
    
Additional features:
- `EndpointSlice`belongs to `Service` of which `endpoint`s are managed
- Same `Endpoint` might be available through multiple `EndpointSlice`s

## Back to `Service`... Traffic policies

> Traffic policies allow us to specify how the traffic should be routed between `Node`s

There are two policies one can set:
- External traffic policy - how traffic from __external sources__ is routed (__from outside cluster__)
- Internal traffic policy - how traffic __within-cluster__ is routed

Both of them support two modes via `.spec.externalTrafficPolicy`:
- `Local`:
    - __preserves client source `IP`__
    - Risks potentially imbalanced traffic (more on `load balancing` later)
- `Cluster`

## Discovering services

> __Discovering services specifies how one can find info about `Service` both from `POD` being part of it as well as outside environment__

There are two ways to do that:
- `ENVIRONMENT_VARIABLES`
- `DNS`

### Environment Variables

> When a `POD` is run a set of `envvars` is automatically added describing `Service` which it belongs to

For example, the `Service` `redis-master` which exposes TCP port `6379` and has been allocated cluster IP address `10.0.0.11`, produces the following environment variables:

In [ ]:
REDIS_MASTER_SERVICE_HOST=10.0.0.11
REDIS_MASTER_SERVICE_PORT=6379
REDIS_MASTER_PORT=tcp://10.0.0.11:6379
REDIS_MASTER_PORT_6379_TCP=tcp://10.0.0.11:6379
REDIS_MASTER_PORT_6379_TCP_PROTO=tcp
REDIS_MASTER_PORT_6379_TCP_PORT=6379
REDIS_MASTER_PORT_6379_TCP_ADDR=10.0.0.11

One can use the above values to act accordingly to `Service` handling the request.

### DNS

> __One should always set up `DNS` service for `k8s` clusters__

One can describe `DNS` as:

> __Phonebook of the internet. Given a name, appropriate `IP` address is returned__

If `DNS` is enabled cluster gains the following capabilities:
- All `POD`s are able to resolve `Service`s based on their name
- One can find `ports` by their name (`DNS SRV` (Service), for example `_http._tcp.my-service`)
- Only way to use `ExternalName` resource (described later)

Later, during configuration lesson we will se how one can configure `CoreDNS` (standard for `DNS` with `k8s`) to work with our cluster

## Publishing Services

> Sometimes (for example frontend) we might want to publish our service to external IP we have (outside of cluster)

There are a few `ServiceType`s one can use to describe the type of the `Service` we want (including the one above):

- `ClusterIP` - __default__, `Service` reachable only from within the cluster
- `NodePort` - exposes `Service` on each of it's `Node`s on a specified `port`. __One can reach `Service` from outside via `<NodeIP>:<NodePort>` FOR EACH NODE__ 
- `LoadBalancer` - Exposes the `Service` externally using a cloud provider's load balancer
- `ExternalName` - Maps the `Service` to the contents of the `externalName` field (e.g. `foo.bar.example.com`)

Example of `NodePort` could be:

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  type: NodePort
  selector:
    app: MyApp
  ports:
      # By default and for convenience, the `targetPort` is set to the same value as the `port` field.
    - port: 80
      targetPort: 80
      # Optional field
      # By default and for convenience, the Kubernetes control plane will allocate a port from a range (default: 30000-32767)
      nodePort: 30007

### LoadBalancer

> `LoadBalancer` is a way to (more or less) uniformly distribute requests across `Node`s

These are often provided by cloud providers (one can also specify a different one via [`.spec.loadBalancerClass`](https://kubernetes.io/docs/concepts/services-networking/service/#load-balancer-class).

> __We will later talk about `Ingress` where we will see a little more about this concept__

By default `Ingress` will be used and it will be presented in `k8s` added `.status` field (__this one, once again, is not specified__):

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
spec:
  selector:
    app: MyApp
  ports:
    - protocol: TCP
      port: 80
      targetPort: 9376
  clusterIP: 10.0.171.239
  type: LoadBalancer
status:
  loadBalancer:
    ingress:
    - ip: 192.0.2.127

For cloud environments, one can specify `loadBalancer` via `metadata.annotations` (we will talk about them in more detail later) within your `Service` configuration.

Larger list can be found [here](https://kubernetes.io/docs/concepts/services-networking/service/#internal-load-balancer):

In [ ]:
[...]
metadata:
    name: my-service
    annotations:
        service.beta.kubernetes.io/aws-load-balancer-internal: "true"
[...]

### ExternalName

Code below maps service to external `DNS` name:

In [ ]:
apiVersion: v1
kind: Service
metadata:
  name: my-service
  namespace: prod
spec:
  type: ExternalName
  externalName: my.database.example.com

# Ingress

> `Kubernetes` object that manages external access to the services in a cluster

![](./images/ingress-one-service.png)

A few features:
- exposes `HTTP`/`HTTPS` from outside cluster to `Service`s within cluster
- may provide `load balancing`
- may provide `SSL Termination` (decryption of data sent to cluster via `SSL`)

One has to install `ingress controller` of some sorts, available options are described [here](https://kubernetes.io/docs/concepts/services-networking/ingress-controllers/).

## Defining resource

As per usual, `.yaml` config:

In [ ]:
---
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: minimal-ingress
  annotations:
    nginx.ingress.kubernetes.io/rewrite-target: /
spec:
  rules:
  - http:
      paths:
      - path: /testpath
        pathType: Prefix
        backend:
          service:
            name: test
            port:
              number: 80

Let's talk about these arguments in more detail

> ### ADD THIS CONTENT: https://kubernetes.io/docs/concepts/services-networking/ingress/#the-ingress-resource

# Network Policies

> ### Describe Network Policies from [here](https://kubernetes.io/docs/concepts/services-networking/network-policies/)

# Challenges

## Mandatory

- What is [`HeadlessService`](https://kubernetes.io/docs/concepts/services-networking/service/#headless-services) and why might we need one?
- What are [Topology Aware Hints](https://kubernetes.io/docs/concepts/services-networking/topology-aware-hints/) and how can the improve our application's responsiveness and performance?


## Additional

- One can use `Service` without `.spec.selector`. Why would we want that? Check out [this part of documentation](https://kubernetes.io/docs/concepts/services-networking/service/#services-without-selectors) for more information
- How are `EndpointSlices` updated via `control plane`? Check [this part of documentation](https://kubernetes.io/docs/concepts/services-networking/endpoint-slices/#distribution-of-endpointslices). Are new `EndpointSlices` created or updated when new `POD`s join the `Service`? 
- Check out how signal goes through `k8s` infrastructure up to the `POD` [here](https://kubernetes.io/docs/concepts/services-networking/service/#proxy-mode-userspace). What's the difference between [user space mode](https://kubernetes.io/docs/concepts/services-networking/service/#proxy-mode-userspace), the one based on [`ip tables`](https://kubernetes.io/docs/concepts/services-networking/service/#proxy-mode-iptables) and [`IPVS` proxy mode](https://kubernetes.io/docs/concepts/services-networking/service/#proxy-mode-ipvs)? How `kube-proxy` chooses which `POD` to choose for traffic in first second or third case?
- What's [Round Robin Scheduling](https://en.wikipedia.org/wiki/Round-robin_scheduling? and how is it used in `k8s`? 